In [1]:
# DFS running combat

In [2]:
import multiprocessing

In [3]:
from DFS_World_States import world, world_grid_states, Weapon
from DFS_Universal_Rules import entity, theoretical_turn_length, subaction_dict, target_distance_scores
from DFS_Action_Series import RuleBasedSequenceDFS, ParallelizedRuleBasedSequenceDFS, action_rules 
from DFS_Location_Series import RuleBasedLocationSequenceDFS2, location_rules
from DFS_Functions import precalc_reward_series, analyze_reward_distribution, post_loc_series_reward_calc, post_obj_reward_series_calc, create_act_loc_obj_rew, post_obj_reward_series_calc2
from DFS_Object_Series import RuleBasedObjectSequenceDFS, object_rules

In [4]:
stage = world(11)

stage.generate_map()

actor = entity(stage)

stage.add_enemy((1,1))
stage.add_enemy((-2,0))
stage.add_enemy((0,3))
#stage.add_enemy((2,2))
#stage.add_enemy((-3,-3))
#stage.add_enemy((1,-1))

stage.add_coin((3,0))
stage.add_coin((-1,-1))
stage.add_coin((1,1))

stage.add_item_to_inventory(actor, Weapon('dagger', 'weapon', 1, 1, 0))

stage.add_weapon_dagger((-2,-1))
stage.add_weapon_dagger((-1,-2))
stage.add_weapon_greataxe((2,0))


In [5]:
stage.grid2

[[[0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [<DFS_World_States.enemy at 0x2110a1192b0>], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []]],
 [[0, 1, 0, 0, 0, [], []],
  [0,
   1,
   0,
   0,
   0,
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []]],
 [[0, 1, 0, 0, 0, [], [<DFS_World_States.Weapon at 0x21129331cd0>]],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []],
  [0, 1, 0, 0, 0, [], []]],
 [[0, 1, 0, 0, 0, []

In [6]:
action_sequence_generator = RuleBasedSequenceDFS(min_length=0,max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
action_series_full_list = action_sequence_generator.generate_sequences()
action_series_full_list

#action_sequence_generator = ParallelizedRuleBasedSequenceDFS(min_length=0, max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
#action_series_full_list = action_sequence_generator.generate_sequences()
#action_series_full_list

[[],
 [0],
 [0, 4],
 [0, 4, 0],
 [0, 4, 0, 5],
 [0, 4, 0, 5, 0],
 [0, 4, 0, 5, 0, 19],
 [0, 4, 0, 5, 0, 19, 0],
 [0, 4, 0, 5, 0, 19, 0, 20],
 [0, 4, 0, 5, 0, 19, 0, 20, 0],
 [0, 4, 0, 5, 0, 19, 0, 20, 0, 19],
 [0, 4, 0, 5, 0, 19, 0, 20, 1],
 [0, 4, 0, 5, 0, 19, 0, 20, 1, 19],
 [0, 4, 0, 5, 0, 19, 0, 20, 19],
 [0, 4, 0, 5, 0, 19, 0, 20, 19, 0],
 [0, 4, 0, 5, 0, 19, 20],
 [0, 4, 0, 5, 0, 19, 20, 0],
 [0, 4, 0, 5, 0, 19, 20, 0, 19],
 [0, 4, 0, 5, 0, 19, 20, 0, 19, 0],
 [0, 4, 0, 5, 0, 19, 20, 1],
 [0, 4, 0, 5, 0, 19, 20, 1, 19],
 [0, 4, 0, 5, 0, 19, 20, 2],
 [0, 4, 0, 5, 0, 19, 20, 2, 19],
 [0, 4, 0, 5, 0, 19, 20, 19],
 [0, 4, 0, 5, 0, 19, 20, 19, 0],
 [0, 4, 0, 5, 1],
 [0, 4, 0, 5, 1, 19],
 [0, 4, 0, 5, 1, 19, 0],
 [0, 4, 0, 5, 1, 19, 0, 20],
 [0, 4, 0, 5, 1, 19, 0, 20, 0],
 [0, 4, 0, 5, 1, 19, 0, 20, 0, 19],
 [0, 4, 0, 5, 1, 19, 0, 20, 19],
 [0, 4, 0, 5, 1, 19, 20],
 [0, 4, 0, 5, 1, 19, 20, 0],
 [0, 4, 0, 5, 1, 19, 20, 0, 19],
 [0, 4, 0, 5, 1, 19, 20, 1],
 [0, 4, 0, 5, 1, 19, 20, 1, 19]

In [7]:
reward_series_full_list = precalc_reward_series(action_series_full_list, actor)
reward_series_full_list

[1,
 1,
 2,
 3,
 5,
 5,
 3,
 3,
 3,
 2,
 1,
 2,
 1,
 1,
 1,
 4,
 3,
 1,
 1,
 3,
 1,
 3,
 1,
 2,
 1,
 5,
 3,
 3,
 3,
 2,
 1,
 1,
 4,
 3,
 1,
 3,
 1,
 2,
 1,
 5,
 3,
 4,
 3,
 1,
 2,
 5,
 3,
 4,
 2,
 3,
 3,
 4,
 3,
 1,
 1,
 3,
 1,
 3,
 1,
 2,
 1,
 3,
 4,
 3,
 1,
 3,
 1,
 2,
 1,
 4,
 4,
 2,
 1,
 4,
 2,
 1,
 4,
 2,
 4,
 2,
 3,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 4,
 2,
 2,
 2,
 1,
 0,
 0,
 3,
 2,
 0,
 2,
 0,
 1,
 0,
 4,
 2,
 3,
 2,
 0,
 1,
 4,
 2,
 3,
 1,
 2,
 2,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 3,
 2,
 0,
 2,
 0,
 1,
 0,
 3,
 3,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 3,
 1,
 2,
 1,
 1,
 4,
 4,
 2,
 2,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 4,
 2,
 2,
 2,
 1,
 0,
 0,
 3,
 2,
 0,
 2,
 0,
 1,
 0,
 4,
 2,
 3,
 2,
 0,
 1,
 4,
 2,
 3,
 1,
 2,
 2,
 3,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 3,
 2,
 0,
 2,
 0,
 1,
 0,
 3,
 3,
 1,
 0,
 3,
 1,
 0,
 3,
 1,
 3,
 1,
 2,
 1,
 1,
 4,
 4,
 2,
 2,
 2,
 1,
 0,
 1,
 0,


In [8]:
analyze_reward_distribution(reward_series_full_list, action_series_full_list)

Total count: 2952711
High: 6
Low: -2.5

Reward Distribution:
% of -2.5s: 0.00%
% of -2.0s: 0.45%
% of -1.5s: 1.29%
% of -1s: 1.50%
% of -0.5s: 6.81%
% of 0s: 7.00%
% of 0.5s: 12.48%
% of 1s: 13.77%
% of 1.5s: 14.00%
% of 2s: 15.77%
% of 2.5s: 9.55%
% of 3s: 12.19%
% of 3.5s: 1.80%
% of 4s: 3.12%
% of 4.5s: 0.07%
% of 5s: 0.19%
% of 6s: 0.00%

Total Qualifiers: 5749



In [9]:
location_series_generator = RuleBasedLocationSequenceDFS2(action_series_list=action_series_full_list,
                                                             reward_series_list=reward_series_full_list,
                                                             location_rules=location_rules,
                                                             acting_entity=actor,
                                                             target_distance_scores=target_distance_scores)
location_series_full_list = location_series_generator.generate_sequences()

In [10]:
post_location_reward_list = post_loc_series_reward_calc(action_series_full_list, location_series_full_list, reward_series_full_list, actor)

In [11]:
print(len(post_location_reward_list))
post_location_reward_list

313982


[([25, 5, 0], [[1, 1], [1, 1]], 5.5),
 ([25, 5, 1], [[1, 1], [-2, 0]], 5.5),
 ([25, 5, 1], [[1, 1], [1, 1]], 5.5),
 ([25, 5, 2], [[1, 1], [-2, 0]], 5.5),
 ([25, 5, 2], [[1, 1], [0, 3]], 5.5),
 ([25, 5, 2], [[1, 1], [1, 1]], 5.5),
 ([25, 5, 3], [[1, 1], [-2, 0]], 5.5),
 ([25, 5, 3], [[1, 1], [0, 3]], 5.5),
 ([25, 5, 3], [[1, 1], [1, 1]], 5.5),
 ([25, 5, 23], [[1, 1], [-2, 0]], 5.5),
 ([25, 5, 23], [[1, 1], [0, 3]], 5.5),
 ([25, 5, 23], [[1, 1], [1, 1]], 5.5),
 ([25, 5, 24], [[1, 1], [-2, 0]], 5.5),
 ([25, 5, 24], [[1, 1], [0, 3]], 5.5),
 ([25, 5, 24], [[1, 1], [1, 1]], 5.5),
 ([4, 13, 0], [[1, 1], [1, 1]], 4.5),
 ([4, 13, 1], [[1, 1], [-2, 0]], 4.5),
 ([4, 13, 1], [[1, 1], [1, 1]], 4.5),
 ([4, 13, 2], [[1, 1], [-2, 0]], 4.5),
 ([4, 13, 2], [[1, 1], [0, 3]], 4.5),
 ([4, 13, 2], [[1, 1], [1, 1]], 4.5),
 ([4, 13, 3], [[1, 1], [-2, 0]], 4.5),
 ([4, 13, 3], [[1, 1], [0, 3]], 4.5),
 ([4, 13, 3], [[1, 1], [1, 1]], 4.5),
 ([4, 13, 23], [[1, 1], [-2, 0]], 4.5),
 ([4, 13, 23], [[1, 1], [0, 3]], 4

In [12]:
# Object Series
object_series_generator = RuleBasedObjectSequenceDFS(post_location_reward_list = post_location_reward_list, 
                                                     target_distance_scores = target_distance_scores,
                                                     object_rules = object_rules, 
                                                     acting_entity = actor)
object_series_full_list = object_series_generator.generate_object_sequences()

print(len(object_series_full_list))

#object_series_full_list

for i in object_series_full_list:
    for j in i:
      print(j)

    print('\n')

object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 1
object series: [25, 5]
object subaction index: 0
object series: [25, 5]
object subaction index: 0
object series: [25, 

IndexError: list index out of range

In [ ]:
# I want the number of items in the object_series_full_list that are not empty
len([i for i in object_series_full_list if i != []])

In [ ]:
post_location_reward_list

In [ ]:
object_series_full_list

In [15]:
#act_loc_obj_rew = create_act_loc_obj_rew(post_location_reward_list, object_series_full_list, actor)
#act_loc_obj_rew

In [ ]:
post_object_reward_list = post_obj_reward_series_calc2(object_series_full_list, actor)
post_object_reward_list

In [17]:
#post_object_reward_list = post_obj_reward_series_calc(action_series_full_list = action_series_full_list, 
#                                                      location_series_full_list = location_series_full_list, 
#                                                      object_series_full_list = object_series_full_list, 
#                                                      reward_series_full_list = reward_series_full_list, 
#                                                      acting_entity = actor)

# this one will include how much damage is being dealt
# if an effective weapon is equipped
# how many coins are picked up

In [18]:
# entity series list???
# in order to specify which entity is being targeted

# shortcuts:
# - currently based on the location series, if there is only one entity in a space, the entity is automatically targeted
# - only if there are multiple entities in a space, the entity series is used to determine the target

In [19]:
# processing a round of combat that includes the queue of actions and possible reactions other entities take

In [20]:
# what needs to happen before I can be running combat?
# - the function to process a creature's turn needs to be done
# - the function to process everyone else's reaction needs to be operational
# - enemies need to be able to take their turn, currently they operate as a form of static entity



In [ ]:
# what needs to happen before the next stage?
# - combat situations needs to be randomly generated
# - combat scenarios need to be ran millions of times